In [12]:
import numpy as np
import pandas as pd

from nltk import word_tokenize
import nltk

import gensim
import pickle

In [13]:
a = nltk.corpus.stopwords.words("spanish")

In [14]:
with open("../modelo/files/stopwords.pkl", "wb") as file:
    pickle.dump(a, file)

In [15]:
detalles = pd.read_json("../scraper_siglas-uc/outputs/detalles.json")
det = detalles.drop(columns = "docente").drop_duplicates() 

In [18]:
det.to_json("../scraper_siglas-uc/outputs/detalles_sp.json", orient = "table", index = False)

In [16]:
data = pd.read_json("../scraper_siglas-uc/outputs/programas_clean.json", orient="table")

programas   = data["programa"].apply(lambda x: word_tokenize(x)).to_list()
diccionario = gensim.corpora.Dictionary(programas)
corpus      = [diccionario.doc2bow(programa) for programa in programas]
model       = gensim.models.LsiModel(corpus, 180, diccionario)
index       = gensim.similarities.MatrixSimilarity(model[corpus])
data.drop(columns = "programa", inplace = True)
index.save('files/index.index')
diccionario.save("files/diccionario.dict")

In [17]:
model.save("files/model.model")

In [94]:
gensim.corpora.MmCorpus.serialize("files/bow_corpus.mm", corpus)

In [91]:
consulta = "programacion en python"
consulta = word_tokenize(consulta)
consulta_bow = diccionario.doc2bow(consulta)
data["score"] = index[model[consulta_bow]]
datos_consulta = det.merge(data, how = "right", on = ["escuela", "sigla"]).sort_values("score", ascending = False).iloc[:, [-1] + [0, 1, 2, 3, 4, 5]].assign(score = lambda x: round(100 * (0.5 * x.score + 0.5), 2))

In [92]:
datos_consulta.head()

,score,escuela,campus,formato,sigla,nombre,creditos
403,93.160004,Ingeniería,San Joaquín,Presencial,ICS2121,Métodos de Optimización,10.0
402,84.459999,Ingeniería,San Joaquín,Presencial,ICS3151,Optimización Bajo Incertidumbre,10.0
628,76.180000,Economía y Administración,San Joaquín,Presencial,EAE4105,Métodos Matemáticos para Economía,10.0
269,76.169998,Matemáticas,San Joaquín,Presencial,EPG3601,Computacion Estadistica Avanzada,15.0
644,73.739998,Economía y Administración,San Joaquín,Presencial,EAA251A,Metodos de Optimizacion,10.0
